In [11]:
import sys
sys.path.append('/home/zhw272/programming/pipeline_utility')
from notebooks.Will.toolbox.IOs.get_bilis_json_file import *
from utilities.alignment.align_point_sets import align_point_sets
from utilities.brain_specimens.get_com import get_atlas_com_dict

## First we check that the coms in Bili/old/data is Kui's DK52 coms transformed with image to image affine transform calculated using airlab

In [2]:
prepi = 'DK39'
billi_dk52_to_DK39_dict = get_transformed_com_dict(prepi)
kui_com = get_kui_dk52_com()

In [3]:
billi_dk52_to_DK39_dict.keys() == kui_com.keys()

True

In [4]:
transform = get_tranformation(prepi)
mov_coms = get_kui_dk52_com()
fix_coms = {}
for name, com in mov_coms.items():
    com = np.array(com, dtype=float)
    fix_coms[name] = transform.forward_point(com).tolist()

In [5]:
billi_dk52_to_DK39_dict == fix_coms

True

## this seems to be the case, now let's try aligning these transformed coms to atlas using code that ed has written

In [6]:
from fixes.add_aligned_com import *

In [48]:
rotation, translation = get_transformation_matrix(prepi, 'corrected')

In [14]:
atlas_centers = get_atlas_com_dict()
shared_structures = set(atlas_centers.keys())&set(billi_dk52_to_DK39_dict.keys())
billi_52_to_39_np = np.array([billi_dk52_to_DK39_dict[structure] for structure in shared_structures])
billi_52_to_39_np = billi_52_to_39_np@np.diag([0.325,0.325,20])
atlas_centers = np.array([atlas_centers[structure] for structure in shared_structures])


In [34]:
rotation,translation = align_point_sets(moving_points=billi_52_to_39_np.T,still_points=atlas_centers.T)
translation = translation/ np.array([0.325,0.325,20]).reshape([3,1])

In [49]:
transformed_com = {}
for landmark,com in billi_dk52_to_DK39_dict.items():
    if rotation is not None:
        scan_run = session.query(ScanRun).filter(ScanRun.prep_id == prepi).one()
        brain_coords = np.asarray(com)
        brain_scale = [scan_run.resolution, scan_run.resolution, 20]
        transformed_com[landmark] = brain_to_atlas_transform(brain_coords, rotation, translation, brain_scale=brain_scale)

In [39]:
from utilities.model.center_of_mass import CenterOfMass
from sql_setup import session
def query_brain_coms(brain, person_id=28, input_type_id=4):
    # default: person is bili, input_type is aligned
    rows = session.query(CenterOfMass)\
        .filter(CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == brain)\
        .filter(CenterOfMass.person_id == person_id)\
        .filter(CenterOfMass.input_type_id == input_type_id)\
        .all()
    row_dict = {}
    for row in rows:
        structure = row.structure.abbreviation
        row_dict[structure] = np.array([row.x, row.y, row.section])
    return row_dict

In [40]:
 bili_aligned_com_database = query_brain_coms(prepi,person_id=1,input_type_id=4)

In [50]:
bili_aligned_com_database['10N_L'],transformed_com['10N_L']

(array([668.536, 353.895, 114.997]),
 array([683.92296748, 380.62047213, 103.18591044]))

In [26]:
len(bili_aligned_com_database),len(transformed_com)

(44, 44)

In [37]:
transformed_com

{'10N_L': array([683.92296748, 380.62047213, 103.18591044]),
 '10N_R': array([798.3368555 , 417.02491237, 137.79448162]),
 '12N': array([775.40569323, 425.16730863, 123.21083545]),
 '3N_L': array([430.952494  , 307.08368797, 132.51701068]),
 '3N_R': array([434.83718771, 315.60759802, 140.23680938]),
 '4N_L': array([475.64353269, 312.44408954, 135.42982174]),
 '4N_R': array([475.61871218, 313.12247789, 135.33276428]),
 '5N_L': array([434.50821091, 387.00825827,  63.46702891]),
 '5N_R': array([600.06114129, 531.20509979, 190.79004409]),
 '6N_L': array([590.52291069, 480.70987016, 136.91001137]),
 '6N_R': array([542.91159329, 433.24742898,  99.86143635]),
 '7N_L': array([597.14453657, 551.42345252,  36.86000051]),
 '7N_R': array([360.90973911, 392.08339648, 121.37223188]),
 'AP': array([737.66325299, 415.4187538 , 116.69120271]),
 'Amb_L': array([602.10318766, 466.5569262 ,  53.98264432]),
 'Amb_R': array([725.79130161, 624.93764372, 173.47486466]),
 'DC_L': array([458.49363379, 270.71457

In [36]:
bili_aligned_com_database

{'10N_L': array([668.536, 353.895, 114.997]),
 '10N_R': array([774.544, 368.762, 153.444]),
 '12N': array([757.562, 381.363, 138.429]),
 '3N_L': array([406.494, 322.039, 132.329]),
 '3N_R': array([409.358, 329.341, 139.998]),
 '4N_L': array([449.512, 319.728, 137.05 ]),
 '4N_R': array([449.61 , 320.404, 136.959]),
 '5N_L': array([439.971 , 403.727 ,  66.1632]),
 '5N_R': array([585.474, 510.87 , 196.933]),
 '6N_L': array([583.687, 465.825, 144.193]),
 '6N_R': array([541.106, 429.15 , 106.136]),
 '7N_L': array([627.196 , 539.662 ,  48.1463]),
 '7N_R': array([354.137, 417.74 , 119.062]),
 'AP': array([721.589, 378.409, 130.491]),
 'Amb_L': array([615.269, 454.56 ,  64.457]),
 'Amb_R': array([724.586, 583.007, 185.974]),
 'DC_L': array([455.005 , 287.266 ,  36.3045]),
 'DC_R': array([794.517, 499.914, 225.335]),
 'IC': array([274.613, 173.149, 163.119]),
 'LC_L': array([485.556 , 325.062 ,  87.8158]),
 'LC_R': array([567.662, 382.394, 168.028]),
 'LRt_L': array([807.198 , 535.841 ,  70.085